< [Hello World](../ica01/hello_world.ipynb) | Contents (TODO) | [Data Mining](../ica03/Data_Mining.ipynb) >

<a href="https://colab.research.google.com/github/stephenbaek/bigdata/blob/master/in-class-assignments/ica02/How_to_Read_and_Represent_Data.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

# How to Read and Represent Data

## 1. Introduction to Pandas

In the previous notebook, we briefly covered the native Python `list` object, which provides a flexible, multi-type container that comes at the cost of efficiency. We also introduced `NumPy` and its `ndarray` object, which, in contrast to `list` object, provides more efficient storage and manipulation, but a less flexible, single-type container.

In this notebook, we will introduce `Pandas` library, a package built on top of `NumPy`, and will build upon the knowledge developed from the previous notebook by looking into the details of `DataFrame` container of `Pandas`. `DataFrame`s are essentially multidimensional arrays with attached row and column labels, and often with heterogeneous types and/or missing data. So, kinda sorta similar to 2D `list` object, with row and column labels. However, `DataFrame` also offers a convenient storage interface for labeled data and implements a number of powerful data operations familiar to users of both database frameworks and spreadsheet programs.

As we saw, NumPy's `ndarray` data structure provides essential features for the type of clean, well-organized data typically seen in numerical computing tasks. While it serves this purpose very well, its limitations become clear when we need more flexibility (e.g., attaching labels to data, working with missing data, etc.) and when attempting operations that do not map well to element-wise broadcasting (e.g., groupings, pivots, etc.), each of which is an important piece of analyzing the less structured data available in many forms in the world around us. Pandas, and in particular its `Series` and `DataFrame` objects, builds on the NumPy array structure and provides efficient access to these sorts of "data munging" tasks that occupy much of a data scientist's time.

In this notebook, we will focus on the mechanics of using `Series`, `DataFrame`, and related structures effectively.
We will use examples drawn from real datasets where appropriate, but these examples are not necessarily the focus.

Also, I hereby acknowledge that many of the codes and explanations I used in this notebook are originally from [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas. The original codes and contents are available on their [GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).

### 1.1. Getting Started with Pandas

Installation of Pandas is as simple as the line of code below:

In [ ]:
!pip install pandas

Once the installation is done, you can import it and check the version:

In [ ]:
import pandas
pandas.__version__

Just as it is a customary to import NumPy with an alias `np`, people typically ipmort Pandas with an alias `pd`:

In [ ]:
import numpy as np
import pandas as pd

### 1.2. Basics of Pandas Objects
At the very basic level, Pandas objects can be thought of as enhanced versions of NumPy structured arrays in which the rows and columns are identified with labels rather than simple integer indices.
As we will see during the course of this notebook, Pandas provides a host of useful tools, methods, and functionality on top of the basic data structures, but nearly everything that follows will require an understanding of what these structures are.
Thus, before we go any further, let's introduce these three fundamental Pandas data structures: the `Series`, `DataFrame`, and `Index`.

### 1.3. Series Object
A Pandas `Series` is a one-dimensional array of indexed data. It can be created from a list or array as follows:

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0])
data

As we see in the output, the ``Series`` wraps both a sequence of values and a sequence of indices, which we can access with the ``values`` and ``index`` attributes.
The ``values`` are simply a familiar NumPy array:

In [ ]:
data.values

The ``index`` is an array-like object of type ``pd.Index``, which we'll discuss in more detail momentarily.

In [ ]:
data.index

Like with a NumPy array, data can be accessed by the associated index via the familiar Python square-bracket notation:

In [ ]:
data[1]

In [ ]:
data[1:3]

As we will see, though, the Pandas ``Series`` is much more general and flexible than the one-dimensional NumPy array that it emulates.

#### `Series` as generalized NumPy array
From what we've seen so far, it may look like the ``Series`` object is basically interchangeable with a one-dimensional NumPy array.
The essential difference is the presence of the index: while the Numpy Array has an *implicitly defined* integer index used to access the values, the Pandas ``Series`` has an *explicitly defined* index associated with the values.

This explicit index definition gives the ``Series`` object additional capabilities. For example, the index need not be an integer, but can consist of values of any desired type.
For example, if we wish, we can use strings as an index:

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

And the item access works as expected:

In [ ]:
data['b']

We can even use non-contiguous or non-sequential indices:

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=[2, 5, 3, 7])
data

In [ ]:
data[5]

#### Series as specialized dictionary

In this way, you can think of a Pandas ``Series`` a bit like a specialization of a Python dictionary.
A dictionary is a structure that maps arbitrary keys to a set of arbitrary values, and a ``Series`` is a structure which maps typed keys to a set of typed values.
This typing is important: just as the type-specific compiled code behind a NumPy array makes it more efficient than a Python list for certain operations, the type information of a Pandas ``Series`` makes it much more efficient than Python dictionaries for certain operations.

The ``Series``-as-dictionary analogy can be made even more clear by constructing a ``Series`` object directly from a Python dictionary:

In [ ]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

By default, a ``Series`` will be created where the index is drawn from the sorted keys.
From here, typical dictionary-style item access can be performed:

In [ ]:
population['California']

Unlike a dictionary, though, the ``Series`` also supports array-style operations such as slicing:

In [ ]:
population['California':'New York']

#### Constructing Series objects

We've already seen a few ways of constructing a Pandas ``Series`` from scratch; all of them are some version of the following:

```python
>>> pd.Series(data, index=index)
```

where ``index`` is an optional argument, and ``data`` can be one of many entities.

For example, ``data`` can be a list or NumPy array, in which case ``index`` defaults to an integer sequence:

In [ ]:
pd.Series([2, 4, 6])

``data`` can be a scalar, which is repeated to fill the specified index:

In [ ]:
pd.Series(5, index=[100, 200, 300])

``data`` can be a dictionary, in which ``index`` defaults to the sorted dictionary keys:

In [ ]:
pd.Series({2:'a', 1:'b', 3:'c'})

In each case, the index can be explicitly set if a different result is preferred:

In [ ]:
pd.Series({2:'a', 1:'b', 3:'c'}, index=[3, 2])

Notice that in this case, the ``Series`` is populated only with the explicitly identified keys.

### 1.4. DataFrame Object

The next fundamental structure in Pandas is the ``DataFrame``.
Like the ``Series`` object discussed in the previous section, the ``DataFrame`` can be thought of either as a generalization of a NumPy array, or as a specialization of a Python dictionary.
We'll now take a look at each of these perspectives.

#### DataFrame as a generalized NumPy array
If a ``Series`` is an analog of a one-dimensional array with flexible indices, a ``DataFrame`` is an analog of a two-dimensional array with both flexible row indices and flexible column names.
Just as you might think of a two-dimensional array as an ordered sequence of aligned one-dimensional columns, you can think of a ``DataFrame`` as a sequence of aligned ``Series`` objects.
Here, by "aligned" we mean that they share the same index.

To demonstrate this, let's first construct a new ``Series`` listing the area of each of the five states discussed in the previous section:

In [ ]:
area_dict = {'California': 423967, 'Texas': 695662, 'New York': 141297,
             'Florida': 170312, 'Illinois': 149995}
area = pd.Series(area_dict)
area

Now that we have this along with the ``population`` Series from before, we can use a dictionary to construct a single two-dimensional object containing this information:

In [ ]:
states = pd.DataFrame({'population': population,
                       'area': area})
states

Like the ``Series`` object, the ``DataFrame`` has an ``index`` attribute that gives access to the index labels:

In [ ]:
states.index

Additionally, the ``DataFrame`` has a ``columns`` attribute, which is an ``Index`` object holding the column labels:

In [ ]:
states.columns

Thus the ``DataFrame`` can be thought of as a generalization of a two-dimensional NumPy array, where both the rows and columns have a generalized index for accessing the data.

#### DataFrame as specialized dictionary

Similarly, we can also think of a ``DataFrame`` as a specialization of a dictionary.
Where a dictionary maps a key to a value, a ``DataFrame`` maps a column name to a ``Series`` of column data.
For example, asking for the ``'area'`` attribute returns the ``Series`` object containing the areas we saw earlier:

In [ ]:
states['area']

Notice the potential point of confusion here: in a two-dimesnional NumPy array, ``data[0]`` will return the first *row*. For a ``DataFrame``, ``data['col0']`` will return the first *column*.
Because of this, it is probably better to think about ``DataFrame``s as generalized dictionaries rather than generalized arrays, though both ways of looking at the situation can be useful.

#### Constructing DataFrame objects from a single Series object

A ``DataFrame`` is a collection of ``Series`` objects, and a single-column ``DataFrame`` can be constructed from a single ``Series``:

In [ ]:
pd.DataFrame(population, columns=['population'])

#### Constructing DataFrame objects from a list of dicts

Any list of dictionaries can be made into a ``DataFrame``.
We'll use a simple list comprehension to create some data:

In [ ]:
data = [{'a': i, 'b': 2 * i}
        for i in range(3)]
pd.DataFrame(data)

Even if some keys in the dictionary are missing, Pandas will fill them in with ``NaN`` (i.e., "not a number") values:

In [ ]:
pd.DataFrame([{'a': 1, 'b': 2}, {'b': 3, 'c': 4}])

#### Constructing DataFrame objects from a dictionary of Series objects

As we saw before, a ``DataFrame`` can be constructed from a dictionary of ``Series`` objects as well:

In [ ]:
pd.DataFrame({'population': population,
              'area': area})

#### Constructing DataFrame objects from a two-dimensional NumPy array

Given a two-dimensional array of data, we can create a ``DataFrame`` with any specified column and index names.
If omitted, an integer index will be used for each:

In [ ]:
pd.DataFrame(np.random.rand(3, 2),
             columns=['foo', 'bar'],
             index=['a', 'b', 'c'])

#### Constructing DataFrame objects from a NumPy structured array

A Pandas ``DataFrame`` operates much like a structured array, and can be created directly from one:

In [ ]:
A = np.zeros(3, dtype=[('A', 'i8'), ('B', 'f8')])
A

In [ ]:
pd.DataFrame(A)

### 1.5. Index Object

We have seen here that both the ``Series`` and ``DataFrame`` objects contain an explicit *index* that lets you reference and modify data.
This ``Index`` object is an interesting structure in itself, and it can be thought of either as an *immutable array* or as an *ordered set* (technically a multi-set, as ``Index`` objects may contain repeated values).
Those views have some interesting consequences in the operations available on ``Index`` objects.
As a simple example, let's construct an ``Index`` from a list of integers:

In [ ]:
ind = pd.Index([2, 3, 5, 7, 11])
ind

#### Index as immutable array

The ``Index`` in many ways operates like an array.
For example, we can use standard Python indexing notation to retrieve values or slices:

In [ ]:
ind[1]

In [ ]:
ind[::2]

``Index`` objects also have many of the attributes familiar from NumPy arrays:

In [ ]:
print(ind.size, ind.shape, ind.ndim, ind.dtype)

One difference between ``Index`` objects and NumPy arrays is that indices are immutable–that is, they cannot be modified via the normal means:

In [ ]:
ind[1] = 0

This immutability makes it safer to share indices between multiple ``DataFrame``s and arrays, without the potential for side effects from inadvertent index modification.

#### Index as ordered set

Pandas objects are designed to facilitate operations such as joins across datasets, which depend on many aspects of set arithmetic.
The ``Index`` object follows many of the conventions used by Python's built-in ``set`` data structure, so that unions, intersections, differences, and other combinations can be computed in a familiar way:

In [ ]:
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])

In [ ]:
indA & indB  # intersection

In [ ]:
indA | indB  # union

In [ ]:
indA ^ indB  # symmetric difference

These operations may also be accessed via object methods, for example ``indA.intersection(indB)``.

## 2. Data Indexing and Selection

In the [previous notebook](../ica01/hello_world.ipynb), we looked in detail at methods and tools to access, set, and modify values in NumPy arrays.
These included indexing (e.g., ``arr[2, 1]``), slicing (e.g., ``arr[:, 1:5]``), masking (e.g., ``arr[arr > 0]``), fancy indexing (e.g., ``arr[0, [1, 5]]``), and combinations thereof (e.g., ``arr[:, [1, 5]]``).
Here we'll look at similar means of accessing and modifying values in Pandas ``Series`` and ``DataFrame`` objects.
If you have used the NumPy patterns, the corresponding patterns in Pandas will feel very familiar, though there are a few quirks to be aware of.

We'll start with the simple case of the one-dimensional ``Series`` object, and then move on to the more complicated two-dimesnional ``DataFrame`` object.

### 2.1. Data Selection in Series

As we saw in the previous section, a ``Series`` object acts in many ways like a one-dimensional NumPy array, and in many ways like a standard Python dictionary.
If we keep these two overlapping analogies in mind, it will help us to understand the patterns of data indexing and selection in these arrays.

#### Series as dictionary

Like a dictionary, the ``Series`` object provides a mapping from a collection of keys to a collection of values:

In [ ]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

In [ ]:
data['b']

We can also use dictionary-like Python expressions and methods to examine the keys/indices and values:

In [ ]:
'a' in data

In [ ]:
data.keys()

In [ ]:
list(data.items())

``Series`` objects can even be modified with a dictionary-like syntax.
Just as you can extend a dictionary by assigning to a new key, you can extend a ``Series`` by assigning to a new index value:

In [ ]:
data['e'] = 1.25
data

This easy mutability of the objects is a convenient feature: under the hood, Pandas is making decisions about memory layout and data copying that might need to take place; the user generally does not need to worry about these issues.

#### Series as one-dimensional array

A ``Series`` builds on this dictionary-like interface and provides array-style item selection via the same basic mechanisms as NumPy arrays – that is, *slices*, *masking*, and *fancy indexing*.
Examples of these are as follows:

In [ ]:
# slicing by explicit index
data['a':'c']

In [ ]:
# slicing by implicit integer index
data[0:2]

In [ ]:
# masking
data[(data > 0.3) & (data < 0.8)]

In [ ]:
# fancy indexing
data[['a', 'e']]

Among these, slicing may be the source of the most confusion.
Notice that when slicing with an explicit index (i.e., ``data['a':'c']``), the final index is *included* in the slice, while when slicing with an implicit index (i.e., ``data[0:2]``), the final index is *excluded* from the slice.

#### Indexers: loc, iloc, and ix

These slicing and indexing conventions can be a source of confusion.
For example, if your ``Series`` has an explicit integer index, an indexing operation such as ``data[1]`` will use the explicit indices, while a slicing operation like ``data[1:3]`` will use the implicit Python-style index.

In [ ]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

In [ ]:
# explicit index when indexing
data[1]

In [ ]:
# implicit index when slicing
data[1:3]

Because of this potential confusion in the case of integer indexes, Pandas provides some special *indexer* attributes that explicitly expose certain indexing schemes.
These are not functional methods, but attributes that expose a particular slicing interface to the data in the ``Series``.

First, the ``loc`` attribute allows indexing and slicing that always references the explicit index:

In [ ]:
data.loc[1]

In [ ]:
data.loc[1:3]

The ``iloc`` attribute allows indexing and slicing that always references the implicit Python-style index:

In [ ]:
data.iloc[1]

In [ ]:
data.iloc[1:3]

A third indexing attribute, ``ix``, is a hybrid of the two, and for ``Series`` objects is equivalent to standard ``[]``-based indexing.
The purpose of the ``ix`` indexer will become more apparent in the context of ``DataFrame`` objects, which we will discuss in a moment.

One guiding principle of Python code is that "explicit is better than implicit."
The explicit nature of ``loc`` and ``iloc`` make them very useful in maintaining clean and readable code; especially in the case of integer indexes, I recommend using these both to make code easier to read and understand, and to prevent subtle bugs due to the mixed indexing/slicing convention.

### 2.2. Data Selection in DataFrame

Recall that a ``DataFrame`` acts in many ways like a two-dimensional or structured array, and in other ways like a dictionary of ``Series`` structures sharing the same index.
These analogies can be helpful to keep in mind as we explore data selection within this structure.

#### DataFrame as a dictionary

The first analogy we will consider is the ``DataFrame`` as a dictionary of related ``Series`` objects.
Let's return to our example of areas and populations of states:

In [ ]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
data

The individual ``Series`` that make up the columns of the ``DataFrame`` can be accessed via dictionary-style indexing of the column name:

In [ ]:
data['area']

Equivalently, we can use attribute-style access with column names that are strings:

In [ ]:
data.area

This attribute-style column access actually accesses the exact same object as the dictionary-style access:

In [ ]:
data.area is data['area']

Though this is a useful shorthand, keep in mind that it does not work for all cases!
For example, if the column names are not strings, or if the column names conflict with methods of the ``DataFrame``, this attribute-style access is not possible.
For example, the ``DataFrame`` has a ``pop()`` method, so ``data.pop`` will point to this rather than the ``"pop"`` column:

In [ ]:
data.pop is data['pop']

In particular, you should avoid the temptation to try column assignment via attribute (i.e., use ``data['pop'] = z`` rather than ``data.pop = z``).

Like with the ``Series`` objects discussed earlier, this dictionary-style syntax can also be used to modify the object, in this case adding a new column:

In [ ]:
data['density'] = data['pop'] / data['area']
data

This shows a preview of the straightforward syntax of element-by-element arithmetic between ``Series`` objects; we'll dig into this further in [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb).

#### DataFrame as two-dimensional array

As mentioned previously, we can also view the ``DataFrame`` as an enhanced two-dimensional array.
We can examine the raw underlying data array using the ``values`` attribute:

In [ ]:
data.values

With this picture in mind, many familiar array-like observations can be done on the ``DataFrame`` itself.
For example, we can transpose the full ``DataFrame`` to swap rows and columns:

In [ ]:
data.T

When it comes to indexing of ``DataFrame`` objects, however, it is clear that the dictionary-style indexing of columns precludes our ability to simply treat it as a NumPy array.
In particular, passing a single index to an array accesses a row:

In [ ]:
data.values[0]

and passing a single "index" to a ``DataFrame`` accesses a column:

In [ ]:
data['area']

Thus for array-style indexing, we need another convention.
Here Pandas again uses the ``loc``, ``iloc``, and ``ix`` indexers mentioned earlier.
Using the ``iloc`` indexer, we can index the underlying array as if it is a simple NumPy array (using the implicit Python-style index), but the ``DataFrame`` index and column labels are maintained in the result:

In [ ]:
data.iloc[:3, :2]

Similarly, using the ``loc`` indexer we can index the underlying data in an array-like style but using the explicit index and column names:

In [ ]:
data.loc[:'Illinois', :'pop']

The ``ix`` indexer allows a hybrid of these two approaches:

In [ ]:
data.ix[:3, :'pop']

Keep in mind that for integer indices, the ``ix`` indexer is subject to the same potential sources of confusion as discussed for integer-indexed ``Series`` objects.

Any of the familiar NumPy-style data access patterns can be used within these indexers.
For example, in the ``loc`` indexer we can combine masking and fancy indexing as in the following:

In [ ]:
data.loc[data.density > 100, ['pop', 'density']]

Any of these indexing conventions may also be used to set or modify values; this is done in the standard way that you might be accustomed to from working with NumPy:

In [ ]:
data.iloc[0, 2] = 90
data

To build up your fluency in Pandas data manipulation, I suggest spending some time with a simple ``DataFrame`` and exploring the types of indexing, slicing, masking, and fancy indexing that are allowed by these various indexing approaches.

#### Additional indexing conventions

There are a couple extra indexing conventions that might seem at odds with the preceding discussion, but nevertheless can be very useful in practice.
First, while *indexing* refers to columns, *slicing* refers to rows:

In [ ]:
data['Florida':'Illinois']

Such slices can also refer to rows by number rather than by index:

In [ ]:
data[1:3]

Similarly, direct masking operations are also interpreted row-wise rather than column-wise:

In [ ]:
data[data.density > 100]

These two conventions are syntactically similar to those on a NumPy array, and while these may not precisely fit the mold of the Pandas conventions, they are nevertheless quite useful in practice.

## 3. Reading Comma Separated Values (CSV) using Pandas

Pandas provides several useful data input/output (IO) functions for a variety of data formats, such as CSV, Excel, JSON, HTML tables, SQL, SAS, HDF5, and etc. Here, we will take a look at a simple CSV example. Once you get familiar with how to handle CSV files, the other file formats will become quite straightforward. For the exhaustive result of what kind of files Pandas can handle, see: https://pandas.pydata.org/pandas-docs/stable/reference/io.html

### 3.1. Reading Baseball Statistics

I'm a big baseball fan. So I decided to use some baseball statistics data in this example. However, you are more than welcome and, in fact, you are actually encouraged to play with some other data sets, whether it be benchmark machine learning data sets from [UCI ML Repository](http://archive.ics.uci.edu/ml/index.php), your own research experiments, class projects, or such.

The data set we will use in this example is provided by [Chadwick Baseball Bureau](http://chadwick-bureau.com/). There are many csv files in different categories (see [this](https://github.com/chadwickbureau/baseballdatabank/tree/master/core) for the full list). As a starter, we will read people database recorded in `People.csv` file. In Pandas, reading a CSV file is as simple as `pd.read_csv(<file-path>)`, which returns a DataFrame parsing the contents of the CSV file:

In [ ]:
people_df = pd.read_csv('https://github.com/chadwickbureau/baseballdatabank/raw/master/core/People.csv')
people_df.head(5) # this displays the first 5 rows of the dataframe

We can see there are a lot of columns containing different types of information about players. To see the full list of columns, you can call:

In [ ]:
people_df.columns

Now, for fun, let's try to compute the average weight and height of baseball players in history.

In [ ]:
people_df[ ['weight', 'height'] ].mean()

In fact, you can compute not just the average, but also other basic statistics such as median, standard deviation, minimum, and maximum.

In [ ]:
people_df[ ['weight', 'height'] ].median()  # median

In [ ]:
people_df[ ['weight', 'height'] ].std()     # standard deviation

In [ ]:
people_df[ ['weight', 'height'] ].min()     # minimum

In [ ]:
people_df[ ['weight', 'height'] ].max()     # maximum

Or, view the summary statistics.

In [ ]:
people_df[ ['weight', 'height'] ].describe()

Notice that `count`s are different because of missing values in the table. Note also that the summary statistics can be computed only on numerical data, not strings. Hence, when you `describe` the entire DataFrame, you will only see a subset of columns.

In [ ]:
people_df.describe()

### 3.2. Selecting Rows based on Values

You will come across many cases where you need to find rows satisfying certain conditions. For example, let's say you would like to find baseball players whose first name is Steve. To achieve this, we can directly apply logical operators on a column:

In [ ]:
people_df['nameFirst'] == 'Steve'

What it returns is an Index object filled with Boolean values. Since this is an Index object, we can directly use them to access the corresponding columns:

In [ ]:
people_df[ people_df['nameFirst'] == 'Steve' ]

When we select rows based on values, it doesn't have to be just one column. In fact, we can apply multiple conditions on different columns.

In [ ]:
people_df[ (people_df['nameFirst'] == 'Steve') & (people_df['birthState'] == 'IA') ]

### 3.3. Joining DataFrames

Another thing that should be useful in the future is to join two or more DataFrames to create a new DataFrame. For example, in the previous example, we used the people database to see the deomgraphics of baseball players. Now, say, you would like to mash it up with pitching data of the players, which is also available in Chadwick Baseball Bureau data set. If you were to do this manually with Python `list` objects or NumPy `ndarray` objects, it would require a lot of coding. However, Pandas provides some useful pre-defined functions that you can use off the shelf as we will see below. Now, let us first read another CSV file from the baseball data set.

In [ ]:
pitcher_df = pd.read_csv('https://github.com/chadwickbureau/baseballdatabank/raw/master/core/Pitching.csv')
pitcher_df.head(5)

The data set we just loaded contains pitching performance of all the baseball pitchers in Major League Baseball (MLB) in history. It's fine if you are not a baseball fan and don't recognize what all those abbreviations are. We are not going to do any of serious analysis in this notebook, but will simply see how we can play with data using Pandas.

Now that we have the pitching data loaded up, our goal now is to merge it with the people data set. However, here are some problems. First off, I assume, even if you are not a baseball fan, you probably know not all players are pitchers. But the people data set contains all baseball players in different positions than just pitchers, whereas the pitching data set contains only the pitchers. So the names in the pitcher data set should be a subset of the names in the people data set. We can confirm this by running the following code.

In [ ]:
pitcher_ids = pitcher_df['playerID'].unique()
people_ids = people_df['playerID'].unique()

print(len(pitcher_ids), len(people_ids))

Note `.unique()` function removes duplicated rows and returns only the unique set of rows (which, by the way, is another DataFrame).

Another problem is that, even though the total number of players in the pitching data set is smaller than that of the people data set, the total number of rows are actually greater in pitching data set than in people data set:

In [ ]:
print(pitcher_df.shape[0], people_df.shape[0])

This is because the pitching data set contains multiple rows per each player, representing the player's performance in different seasons (years). For example, my childhood hero, [Greg Maddux](https://en.wikipedia.org/wiki/Greg_Maddux) played in MLB from 1986 to 2008 and there are multiple rows in the pitching data set that correspond to those years.

In [ ]:
# search for `playerID` of Greg Maddux
maddux_id = people_df['playerID'][ (people_df['nameLast'] == 'Maddux') & (people_df['nameFirst'] == 'Greg') ]

# print the years where his data is recorded
pitcher_df['yearID'][ pitcher_df['playerID'] == maddux_id.iloc[0] ]

Note that the years 2006 and 2008 have duplications because Greg Maddux played in two different teams in those seasons (Cubs and Dodgers in 2006, Padres and Dodgers in 2008).

Now, these problems might have made you a little scared because merging the two table now seems to be a bit disastrous as we will have to manage all those duplications and stuff. Well, the reason why Pandas is so powerful is because it already comes with pre-defined functions that will save your lives!

In [ ]:
df = pitcher_df.merge(people_df, left_on='playerID', right_on='playerID')
df.head(5)

As in the above example, merging two tables can be as simple as `.merge()` and Pandas will do all the heavy-liftings for you. Only two things you need to provide: the name of the key column in the left table (`pitcher_df` in our case) and the name of the key column in the right table (`people_df` in our case). The function `merge()` will then sort out those keys, reorganize them, and merge the rows based on those key values.

Now that we have two DataFrames mashed up and merged into one big DataFrame, we can study the connection between, e.g., demographic data and pitching performance. For example, for whatever random reason, let's say you want to know the correlation between pitcher's height and the [Earned Run Average (ERA)](https://en.wikipedia.org/wiki/Earned_run_average). With the merged DataFrame `df`, the relevant data can be retrieved as below.

In [ ]:
height_ERA = df[ ['height', 'ERA'] ]  # select the relevant columns
height_ERA

One thing to notice is that there are many missing values in the extracted DataFrame. This can be inspected by calling `isnull()` function.

In [ ]:
height_ERA.isnull()

In many cases, rows with missing values are simply dropped. This can be done by calling `dropna()` function.

In [ ]:
height_ERA = height_ERA.dropna()
height_ERA

We haven't learned any of those fancy machine learning/statistical analysis methods yet (we will get there pretty soon!). For now, let us just try a simple analysis to see if there's any correlation between the height of a pitcher and the ERA by taking the average of ERA for different height groups.

In [ ]:
height_ERA['height'].describe(percentiles=[0.1,0.9])

From above, we can notice that about 10% of MLB pitchers in history are taller than 76 inches and, on the opposite end, 10% of them are shorter than 70 inches. Let us summarize the ERA stats for those who are tall, short, and in the middle.

In [ ]:
# Assignment: compute mean and standard deviation of tall (height > 76 inches) pitchers.


In [ ]:
# Assignment: compute mean and standard deviation of short (height < 70 inches) pitchers.


In [ ]:
# Assignment: compute mean and standard deviation of other (70 inches < height < 76 inches) pitchers.


**Assignment**: Was there any noticeable difference among the three groups? Can you conclude that the height is correlated with the pitcher's ERA? Provide your answer below.

To wrap things up, we introduced Pandas library in this notebook. It certainly is a powerful tool for data analysis and getting familiar with it is a prerequisite these days to become a data scientist. In fact, what I introduced in this notebook is just a tip of the iceberg and there are so many awesome things about Pandas which couldn't be covered in this notebook. Once you get comfortable with the topics above, I strongly encourage you to navigate through the official documentation of Pandas which can be found in this [link](https://pandas.pydata.org/pandas-docs/stable/index.html).

< [Hello World](../ica01/hello_world.ipynb) | Contents (TODO) | [Data Mining](../ica03/Data_Mining.ipynb) >

<a href="https://colab.research.google.com/github/stephenbaek/bigdata/blob/master/in-class-assignments/ica02/How_to_Read_and_Represent_Data.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>